## Preprocessing

In [273]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [274]:
# Drop the EIN column
app_df = application_df.drop(['EIN'], 1)

C:\Users\dskan\AppData\Local\Temp/ipykernel_38400/1257282588.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app_df = application_df.drop(['EIN'], 1)


In [275]:
# Display Counts of Special Considerations to understand if there is parity
app_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [276]:
# Given the small amount of special considerations, drop that column, as well as the status column

app_df = app_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS'], 1)

C:\Users\dskan\AppData\Local\Temp/ipykernel_38400/4021192280.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  app_df = app_df.drop(['STATUS', 'SPECIAL_CONSIDERATIONS'], 1)


In [277]:
app_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [278]:
# Determine the number of unique values in each column.
app_df.nunique()

NAME                19568
APPLICATION_TYPE       17
AFFILIATION             6
CLASSIFICATION         71
USE_CASE                5
ORGANIZATION            4
INCOME_AMT              9
ASK_AMT              8747
IS_SUCCESSFUL           2
dtype: int64

In [279]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = app_df['APPLICATION_TYPE'].value_counts()
print(app_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [280]:
# The bottom 8 app types contain less than 300 entries - bin them
app_types_to_replace = list(app_counts[app_counts < 500].index)
# Replace in dataframe
for app in app_types_to_replace:
    app_df['APPLICATION_TYPE'] = app_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
app_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [281]:
# Look at CLASSIFICATION value counts for binning
class_counts = app_df["CLASSIFICATION"].value_counts()
print(class_counts)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64


In [282]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts[class_counts > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [283]:
# The bottom 25 classifications account for less than 1000 entries and all contain less than 100 entries each - bin them
class_to_replace = list(class_counts[class_counts < 100].index)
# Replace in dataframe
for cls in class_to_replace:
    app_df['CLASSIFICATION'] = app_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [284]:
name_counts = app_df['NAME'].value_counts()
print(name_counts)

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64


In [285]:
name_counts[name_counts < 3]

IOTA PHI LAMBDA SORORITY INC                                            2
DANCE INITIATIVE INC                                                    2
NATIONAL SOCIETY OF TOLE AND DECORATIVE PAINTERS INC                    2
UNITED TRANSPORTATION UNION                                             2
INTERNATIONAL ALLIANCE THEATRICAL STAGE EMPLOYEES                       2
                                                                       ..
ST LOUIS SLAM WOMENS FOOTBALL                                           1
AIESEC ALUMNI IBEROAMERICA CORP                                         1
WEALLBLEEDRED ORG INC                                                   1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI    1
WATERHOUSE CHARITABLE TR                                                1
Name: NAME, Length: 18991, dtype: int64

In [286]:
# there appear to be many companies with multiple requests - understanding that 
# it is likely that an effort will be successful with more than one request, 
# I've chosen to bin all companies with less than 3 requests
names_to_replace = list(name_counts[name_counts < 3].index)
# Replace in dataframe
for cls in names_to_replace:
    app_df['NAME'] = app_df['NAME'].replace(cls,"Other")
    
# Check to make sure binning was successful
app_df['NAME'].value_counts()

Other                                                         19206
PARENT BOOSTER USA INC                                         1260
TOPS CLUB INC                                                   765
UNITED STATES BOWLING CONGRESS INC                              700
WASHINGTON STATE UNIVERSITY                                     492
                                                              ...  
ORDER OF EASTERN STAR PRINCE HALL AFFILIATION OF LOUISIANA        3
UEMPOWERU INC                                                     3
TRUSTEES OF DARTMOUTH COLLEGE                                     3
NATIONAL FELLOWSHIP OF RACEWAY MINISTRIES                         3
WESTERN DRESSAGE ASSOCIATION OF AMERICA                           3
Name: NAME, Length: 578, dtype: int64

In [288]:
app_df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,Other,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,Other,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [289]:
# Convert categorical data to numeric with `pd.get_dummies`
app_dummies_df = pd.get_dummies(app_df)
app_dummies_df.head()

,ASK_AMT,IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,NAME_AFRICAN-AMERICAN POSTAL LEAGUE UNITED FOR SUCCESS A-PLUS,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [290]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
X = app_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = app_dummies_df["IS_SUCCESSFUL"].values
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [291]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [292]:
# Four layers - starting with relu, and hidden layers using sigmoid - 
number_input_features  = len(X_train[0])
layer_one_nodes = 100
layer_two_nodes = 50
layer_three_nodes = 70
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=layer_one_nodes, input_dim=number_input_features, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=layer_two_nodes, input_dim=layer_one_nodes, activation = 'sigmoid'))

# third hidden layer
nn.add(tf.keras.layers.Dense(units=layer_three_nodes, input_dim=layer_two_nodes, activation = 'sigmoid'))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=layer_four_nodes, input_dim=layer_three_nodes, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, input_dim=layer_four_nodes, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_123 (Dense)           (None, 100)               62500     
                                                                 
 dense_124 (Dense)           (None, 50)                5050      
                                                                 
 dense_125 (Dense)           (None, 70)                3570      
                                                                 
 dense_126 (Dense)           (None, 70)                4970      
                                                                 
 dense_127 (Dense)           (None, 1)                 71        
                                                                 
Total params: 76,161
Trainable params: 76,161
Non-trainable params: 0
_________________________________________________________________


In [293]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics="accuracy")

In [296]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 5ms/step - loss: 0.4058 - accuracy: 0.8062
Epoch 2/100
804/804 [==============================] - 6s 7ms/step - loss: 0.4037 - accuracy: 0.8076
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.4037 - accuracy: 0.8077
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4024 - accuracy: 0.8086
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4022 - accuracy: 0.8085
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4016 - accuracy: 0.8085
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4007 - accuracy: 0.8085
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3996 - accuracy: 0.8100
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.3994 - accuracy: 0.8096
Epoch 10/100
804/804 [==============================] - 4s 5ms/step - loss: 0.3993 - accura

804/804 [==============================] - 4s 5ms/step - loss: 0.3864 - accuracy: 0.8149
Epoch 82/100
804/804 [==============================] - 5s 6ms/step - loss: 0.3861 - accuracy: 0.8151
Epoch 83/100
804/804 [==============================] - 4s 5ms/step - loss: 0.3868 - accuracy: 0.8152
Epoch 84/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3862 - accuracy: 0.8154
Epoch 85/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3862 - accuracy: 0.8152
Epoch 86/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3864 - accuracy: 0.8152
Epoch 87/100
804/804 [==============================] - 3s 4ms/step - loss: 0.3863 - accuracy: 0.8153
Epoch 88/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3862 - accuracy: 0.8150
Epoch 89/100
804/804 [==============================] - 3s 3ms/step - loss: 0.3861 - accuracy: 0.8161
Epoch 90/100
804/804 [==============================] - 2s 3ms/step - loss: 0.3857 - accuracy: 

In [297]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4632 - accuracy: 0.7983 - 883ms/epoch - 3ms/step
Loss: 0.4632071852684021, Accuracy: 0.7982507348060608


In [298]:
# Export our model to HDF5 file
#  YOUR CODE GOES HERE
nn.save("alphabetcharity.hd5")

INFO:tensorflow:Assets written to: alphabetcharity.hd5\assets


In [300]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [303]:
rf_model = RandomForestClassifier(n_estimators=256, random_state =156)
rf_model = rf_model.fit(X_train_scaled, y_train)
y_pred = rf_model.predict(X_test_scaled)
print(f"Random Forest Model Accuracy: {accuracy_score(y_test,y_pred)*100:.2f}")

Random Forest Model Accuracy: 77.82
